# Titanic Dataset - One-Hot Vectors

Here, we'll build a model to predict who survived the Titanic disaster. Transform data between numerical and categorical types, including use of one-hot vectors.

## Data prepartion

Open and clean the dataset:

In [13]:
 pip install wget

Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas
#!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/m0c_logistic_regression.py

import wget

url = 'https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/m0c_logistic_regression.py'

filename = wget.download(url)
print(filename)


!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/titanic.csv

# Open our dataset from file
dataset = pd.read_csv("titanic.csv", index_col=False, sep=",", header=0)

# Fill missing cabin information with 'Unknown'
dataset["Cabin"].fillna("Unknown", inplace=True)

# Remove rows missing Age information
dataset.dropna(subset=["Age"], inplace=True)

# Remove the Name, PassengerId, and Ticket fields to read our print-outs
dataset.drop(["PassengerId", "Name", "Ticket"], axis=1, inplace=True)

dataset.head()

100% [..................................................................................] 827 / 827m0c_logistic_regression (3).py


'wget' is not recognized as an internal or external command,
operable program or batch file.


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,Unknown,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,Unknown,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,Unknown,S


## About Our Model

We'll training a model type known as Logistic Regression, which will predict who survives the Titanic disaster.

1. Accepts our data frame and a list of features to include in the model. 
2. Trains the model.
3. Returns a number that states how well the model performs as it predicts passenger survival. **Smaller numbers are better.**

## Numerical Only

Create a model that uses only the numerical features.

Use `Pclass` here as an ordinal feature, rather than a one-hot encoded categorical feature.

In [29]:
from m0c_logistic_regression import train_logistic_regression

features = ["Age", "Pclass", "SibSp", "Parch", "Fare"] 
loss_numerical_only = train_logistic_regression(dataset, features)

print(f"Numerical-Only, Log-Loss (cost): {loss_numerical_only}")

Numerical-Only, Log-Loss (cost): 0.6121682789483452


Let's see if categorical features will improve the model.

## Binary Categorical Features

Categorical features with only two potential values can be encoded in a single column, as `0` or `1`.

Convert `Sex` values into `IsFemale` - a `0` for male and `1` for female - and include that in our model.

In [30]:
# Swap male / female with numerical values
# We can do this because there are only two categories
dataset["IsFemale"] = dataset.Sex.replace({'male':0, 'female':1})

# Print out the first few rows of the dataset
print(dataset.head())

# Run and test the model, also using IsFemale this time
features = ["Age", "Pclass", "SibSp", "Parch", "Fare", "IsFemale"] 
loss_binary_categoricals = train_logistic_regression(dataset, features)

print(f"\nNumerical + Sex, Log-Loss (cost): {loss_binary_categoricals}")

   Survived  Pclass     Sex   Age  SibSp  Parch     Fare    Cabin Embarked  \
0         0       3    male  22.0      1      0   7.2500  Unknown        S   
1         1       1  female  38.0      1      0  71.2833      C85        C   
2         1       3  female  26.0      0      0   7.9250  Unknown        S   
3         1       1  female  35.0      1      0  53.1000     C123        S   
4         0       3    male  35.0      0      0   8.0500  Unknown        S   

  Deck  IsFemale  
0    U         0  
1    C         1  
2    U         1  
3    C         1  
4    U         0  

Numerical + Sex, Log-Loss (cost): 0.4707144738808962


The loss (error) decreased! This model performs better than the previous model.

## One-Hot Encoding

Ticket class (`Pclass`) is an Ordinal feature. Its potential values (1, 2 & 3) have an order and they have equal spacing. However, this even spacing might be incorrect - in stories about the Titanic, the third-class passengers were treated much worse than those in 1st and 2nd class.

Convert `Pclass` into a categorical feature using one-hot encoding:

In [32]:
# Get all possible categories for the "PClass" column
print(f"Possible values for PClass: {dataset['Pclass'].unique()}")

# Use Pandas to One-Hot encode the PClass category
dataset_with_one_hot = pandas.get_dummies(dataset, columns=["Pclass"], drop_first=False)

# Add back in the old Pclass column, for learning purposes
dataset_with_one_hot["Pclass"] = dataset.Pclass

# Print out the first few rows
dataset_with_one_hot.head()

Possible values for PClass: [3 1 2]


,Survived,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Deck,IsFemale,Pclass_1,Pclass_2,Pclass_3,Pclass
0,0,male,22.0,1,0,7.2500,Unknown,S,U,0,0,0,1,3
1,1,female,38.0,1,0,71.2833,C85,C,C,1,1,0,0,1
2,1,female,26.0,0,0,7.9250,Unknown,S,U,1,0,0,1,3
3,1,female,35.0,1,0,53.1000,C123,S,C,1,1,0,0,1
4,0,male,35.0,0,0,8.0500,Unknown,S,U,0,0,0,1,3


Note that `Pclass` converted into three values: `Pclass_1`, `Pclass_2` and `Pclass_3`.

Rows with `Pclass` values of 1 have a value in the `Pclass_1` column. We see the same pattern for values of 2 and 3.

Now, re-run the model, and treat `Pclass` values as a categorical values, rather than ordinal values.

In [33]:
# Run and test the model, also using Pclass as a categorical feature this time
features = ["Age", "SibSp", "Parch", "Fare", "IsFemale",
            "Pclass_1", "Pclass_2", "Pclass_3"]

loss_pclass_categorical = train_logistic_regression(dataset_with_one_hot, features)

print(f"\nNumerical, Sex, Categorical Pclass, Log-Loss (cost): {loss_pclass_categorical}")


Numerical, Sex, Categorical Pclass, Log-Loss (cost): 0.4717111608203336


This seems to have made things slightly worse!

## Including Cabin

Recall that many passengers had `Cabin` information. `Cabin` is a categorical feature and should be a good predictor of survival, because people in lower cabins probably had little time to escape during the sinking.

Let's encode cabin using one-hot vectors, and include it in a model. This time, there are so many cabins that we won't print them all out.

In [34]:
# Use Pandas to One-Hot encode the Cabin and Pclass categories
dataset_with_one_hot = pandas.get_dummies(dataset, columns=["Pclass", "Cabin"], drop_first=False)

# Find cabin column names
cabin_column_names = list(c for c in dataset_with_one_hot.columns if c.startswith("Cabin_"))

# Print out how many cabins there were
print(len(cabin_column_names), "cabins found")

# Make a list of features
features = ["Age", "SibSp", "Parch", "Fare", "IsFemale",
            "Pclass_1", "Pclass_2", "Pclass_3"] + \
            cabin_column_names

# Run the model and print the result
loss_cabin_categorical = train_logistic_regression(dataset_with_one_hot, features)

print(f"\nNumerical, Sex, Categorical Pclass, Cabin, Log-Loss (cost): {loss_cabin_categorical}")

135 cabins found

Numerical, Sex, Categorical Pclass, Cabin, Log-Loss (cost): 0.46003262953858026


That's our best result so far!

## Improving Power

Including very large numbers of categorical classes - for example, 135 cabins - is often not the best way to train a model, because the model only has a few examples of each category class to learn from.

Sometimes, we can improve models if we simplify features. `Cabin` was probably useful because it indicated which Titanic deck people were probably situated in: those in lower decks would have had their quarters flooded first. 

It might become simpler to use deck information, instead of categorizing people into Cabins. 

Let's simplify what we have run, replacing the 135 `Cabin` categories with a simpler `Deck` category that has only 9 values: A to G, T, and U (Unknown)

In [35]:
# We have cabin names, like A31, G45. The letter refers to the deck that
# the cabin was on. Extract just the deck and save it to a column. 
dataset["Deck"] = [c[0] for c in dataset.Cabin]

print("Decks: ", sorted(dataset.Deck.unique()))

# Create one-hot vectors for:
# Pclass - the class of ticket. (This could be treated as ordinal or categorical)
# Deck - the deck that the cabin was on
dataset_with_one_hot = pandas.get_dummies(dataset, columns=["Pclass", "Deck"], drop_first=False)

# Find the deck names
deck_of_cabin_column_names = list(c for c in dataset_with_one_hot.columns if c.startswith("Deck_"))
print(deck_of_cabin_column_names)
 
features = ["Age", "IsFemale", "SibSp", "Parch", "Fare", 
            "Pclass_1", "Pclass_2", "Pclass_3",
            "Deck_A", "Deck_B", "Deck_C", "Deck_D", 
            "Deck_E", "Deck_F", "Deck_G", "Deck_U", "Deck_T"]

loss_deck = train_logistic_regression(dataset_with_one_hot, features)

print(f"\nSimplifying Cabin Into Deck, Log-Loss (cost): {loss_deck}")

Decks:  ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'U']
['Deck_A', 'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 'Deck_T', 'Deck_U']

Simplifying Cabin Into Deck, Log-Loss (cost): 0.45880228707730186


## Comparing Models

Let's compare the `loss` for these models:

In [36]:
# Use a dataframe to create a comparison table of metrics
# Copy metrics from previous Unit

l =[["Numeric Features Only", loss_numerical_only],
    ["Adding Sex as Binary", loss_binary_categoricals],
    ["Treating Pclass as Categorical", loss_pclass_categorical],
    ["Using Cabin as Categorical", loss_cabin_categorical],
    ["Using Deck rather than Cabin", loss_deck]]

pandas.DataFrame(l, columns=["Dataset", "Log-Loss (Low is better)"])

,Dataset,Log-Loss (Low is better)
0,Numeric Features Only,0.612168
1,Adding Sex as Binary,0.470714
2,Treating Pclass as Categorical,0.471711
3,Using Cabin as Categorical,0.460033
4,Using Deck rather than Cabin,0.458802


We can see that including categorical features can both improve and harm how well a model works. Often, experimentation is the best way to find the best model. 

## Summary

Takeaway: How to use One-Hot encoding to address categorical data.

We explored how sometimes critical thinking about the problem at hand can improve a solution more than simply including all possible features in a model.